In [20]:
import numpy as np
import pandas as pd
import functions.multi_occupation_network as multi_occupation_network

# Calibration Excersize 2: One Occupation Per Sector

Suppose there is one type of labor specific to each sector, in other words that $\mathcal{O}=J$. We calibrate our model to mathc BEA input-output tables, and treat the residual, the part of production not accounted for by intermediate inputs, as the labor elasticity. 

In [21]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]
O = J

φ = np.array(dfParam.φ)

epsN = np.diag(np.array(dfParam.α))
epsD = np.array(dfParam.θ_alt).reshape((J,1))

θ = dfLabor_market_yearly['v'] / dfLabor_market_yearly['u'] 
ν = np.ones(J) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ
r = 0.045
curlyT = np.diag(r / (φ*θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.
curlyL = np.eye(J)

First, we define the shocks we are interested in to allow us to easily change these in the future, that is we define the vectors $d\log\bm{A}$ and $d\log\bm{H}$.

In [22]:
dlog_A = np.zeros((J, 1))
dlog_A[3] = 0.01 
dlog_H = np.zeros((O,1))

We assume Cobb-Douglas production and preferences.

In [23]:
dlog_lam = np.zeros_like(dlog_A)
dlog_epsN = np.zeros_like(epsN)
dlog_epsD = np.zeros_like(epsD)

Finally, we need to make some assumption about how wages change. We begin by assuming that nominal wages do not change in respond to changes in either productivity or the labor force.

In [24]:
#epsW_A = np.zeros((O,J))
gamma_A = 1
gamma_H = 1
epsW_A, epsW_H = multi_occupation_network.WageElasticityFunc(gamma_A, gamma_H, Psi, curlyL, epsN)
epsW_A

array([[0.00189839],
       [0.00188326],
       [0.00823666],
       [0.021484  ],
       [0.00165608],
       [0.00103183],
       [0.00201722],
       [0.0019834 ],
       [0.00301888],
       [0.00468031],
       [0.00450622],
       [0.0028292 ],
       [0.00172848],
       [0.00300076],
       [0.001638  ],
       [0.00251799],
       [0.00150024]])

With these pieces in hand, we can now estimate responses to tightness. For details, see the multiple occupations example code.

In [25]:
curlyE = multi_occupation_network.curlyEFunc(dlog_epsN,epsN)
dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN)
dlog_theta

array([[-5.55111512e-17],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 2.77555756e-17],
       [ 8.32667268e-17],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 5.55111512e-17],
       [ 0.00000000e+00],
       [-5.55111512e-17],
       [ 0.00000000e+00],
       [-8.32667268e-17],
       [ 5.55111512e-17],
       [ 0.00000000e+00],
       [ 5.55111512e-17],
       [ 8.32667268e-17]])

In [26]:
num=0
dlog_p = multi_occupation_network.PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, curlyT, curlyL, num=num)
dlog_p

array([[ 0.00000000e+00],
       [ 1.51291126e-05],
       [-6.33827206e-03],
       [-1.95856091e-02],
       [ 2.42313346e-04],
       [ 8.66557770e-04],
       [-1.18828419e-04],
       [-8.50104220e-05],
       [-1.12048795e-03],
       [-2.78191602e-03],
       [-2.60782739e-03],
       [-9.30805219e-04],
       [ 1.69910558e-04],
       [-1.10237000e-03],
       [ 2.60390345e-04],
       [-6.19598947e-04],
       [ 3.98151742e-04]])

In [27]:
dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.00189839],
       [0.00188326],
       [0.00823666],
       [0.021484  ],
       [0.00165608],
       [0.00103183],
       [0.00201722],
       [0.0019834 ],
       [0.00301888],
       [0.00468031],
       [0.00450622],
       [0.0028292 ],
       [0.00172848],
       [0.00300076],
       [0.001638  ],
       [0.00251799],
       [0.00150024]])

We can check labor market clearing holds.

In [28]:
multi_occupation_network.LaborSupply(dlog_H, dlog_theta, curlyF) - multi_occupation_network.LaborDemand(dlog_wR, dlog_y, dlog_p, dlog_epsN, curlyL)

array([[-7.58941521e-18],
       [ 5.85469173e-18],
       [ 5.20417043e-18],
       [ 6.93889390e-18],
       [ 8.67361738e-18],
       [ 2.40692882e-17],
       [ 3.90312782e-18],
       [ 4.33680869e-18],
       [ 2.38524478e-17],
       [ 5.20417043e-18],
       [-1.38777878e-17],
       [ 2.60208521e-18],
       [-1.32272665e-17],
       [ 2.99239800e-17],
       [ 4.33680869e-18],
       [ 2.03830008e-17],
       [ 2.53703308e-17]])

Aggregate output changes are.

In [29]:
dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([[0.00717491]])

Since labor market frictions are the only source of inefficiency in our network economy, in the absence of labor market frictions our network economy is fully efficient. It is well known that Hulten's theorem holds in efficient network economies with Cobb-Douglas technology. We can therefore easily compare the implications of labor market frictions for aggregate output with the implications of the same technology shocks absent labor market frictions.  

In [30]:
epsD.T @ Psi @ dlog_A

array([[0.00717491]])

In [33]:
dlog_wR + curlyL @ dlog_p

array([[0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839],
       [0.00189839]])